In [15]:
import os 
os.chdir('..')  # Path hack: NOTE if you run this cell multiple times, you might need to comment this line out
from utils.data import read_profit_optimization_data
from config import standard_mock_data
from services.classical_optimizers import binary_profit_optimizer, discrete_profit_optimizer
from models.ProfitQubo import ProfitQubo
from neal import SimulatedAnnealingSampler
from dwave.system import LeapHybridDQMSampler

What ever constants we want

In [5]:
budget = 10000

First let's read in our data. We can start off with the small data set

In [4]:
profit, cost = read_profit_optimization_data(standard_mock_data['small'])
# TODO: Somehow visualize the data

Next we try the classical solution. Note, we have a discreet method (which is not very applicable to the real world) and a binary method (more applicable).

discrete method just for completeness (recall: this is not very applicable to real world)

In [8]:
binary_solution, binary_cost, binary_profit = binary_profit_optimizer(profit=profit, cost=cost, budget=budget)
print('Found binary (crude) profit optimization solution', binary_solution, binary_cost, binary_profit)

Found binary (crude) profit optimization solution (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19) 356.73150914723203 499.42411280612487


Next we use the discrete solution

In [9]:
discrete_profit = discrete_profit_optimizer(profit=profit, cost=cost, budget=budget)
print('Found discrete (crude) profit optimization solution', discrete_profit)
# TODO: Somehow visualize the result and make it comparable to the next steps
# TODO: fix this function to make it yield appropriate data (i.e. solution set, cost)

Found discrete (crude) profit optimization solution 14130.2


Great! Now lets use the QUBO formulation. We start with the simulated annealing

In [13]:
# TODO make the ProfitQubo take a list of budgets (i.e. max_number_of_products)
qubo = ProfitQubo(profits=profit, costs=cost, budget=budget, max_number_of_products=10)
sampler = SimulatedAnnealingSampler().sample_dqm
qubo.solve(sampler, **{"num_reads":100, "num_sweeps": 100000})
print(qubo.solution_set)

AttributeError: 'SimulatedAnnealingSampler' object has no attribute 'sample_dqm'

Ok, lets try the hybrid solver now

In [16]:
sampler = LeapHybridDQMSampler().sample_dqm
qubo.solve(sampler)
print(qubo.solution_set)

   x0 x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11 x12 x13 ... y13         energy num_oc.
12 10 10 10 10 10 10 10  9 10 10   9  10  10  10 ...   1   -4205.553113       1
3   0  0  1  0 10  5  8  4 10  0   9   0   8  10 ...   1   -3218.261363       1
22 10 10 10 10 10  8  9  4  2  2  10   2   1   3 ...   1    -3123.25136       1
10  0 10 10 10  0  0  8  4  0  0   7   6  10  10 ...   1   -2893.952026       1
20 10  0 10  2  4  7  8  8  5  9   7   9   0   7 ...   0    -2742.28209       1
4   0  0  1  0  4  5  0  4  0  0   7   0  10  10 ...   1   -2674.632111       1
8   0  0  0  0  0  7  5  0 10  0   7   6  10  10 ...   1   -2585.862854       1
11 10 10 10 10 10 10 10 10 10 10  10  10  10  10 ...   0   -2568.389841       1
13 10 10 10 10 10 10 10 10 10 10  10  10  10  10 ...   0   -2568.389841       1
14  0  0  0  0  0  0  0  0  0  0   6   0  10  10 ...   1   -2541.815244       1
21  0  0  0  0  0  0  0  0  0  0   4   0  10  10 ...   1   -2538.177225       1
19  0  0  0  0  0  0  0  0  0  0   1   0